## Sandbox for testing out chart options

It can be helpful to test out changes to charting code in a Jupyter Notebook before updating the code used in the actual Plotly app, as the notebook format makes it easier to see how tweaks to the code will influence the results.

In [1]:
import platform

In [2]:
platform.node()

'DESKTOP-83K77J1'

In [3]:
ec = ['School', 'Grade', 'Gender',
'Race', 'Ethnicity']

enrollment_comparisons_with_none_option = ec.copy()
enrollment_comparisons_with_none_option.append(None)

In [4]:
enrollment_comparisons_with_none_option

['School', 'Grade', 'Gender', 'Race', 'Ethnicity', None]

In [5]:
color_value = 'None'
secondary_differentiator = None
if color_value == 'None':
    color_value = None

In [6]:
color_value = 'Grade'
comparison_values = ['School']

color_value in comparison_values

False

In [7]:
type(color_value)

str

In [8]:
import pandas as pd
import plotly.express as px

In [9]:
df_curr_enrollment = pd.read_csv('table_students.csv')
df_curr_enrollment

,Student_ID,First_Name,Last_Name,Full_School_Name,School,Grade,Gender,Race,Ethnicity,Street,City,State,Zip,Lat,Lon,Address,geometry,Students,Grade_for_Sorting
0,2403,Donna,Adams,Chestnut Academy,CA,1,Female,Asian,Hispanic,3846 KING ST,ALEXANDRIA,VA,22302,38.831460,-77.094591,"3846 KING ST, ALEXANDRIA, VA 22302",POINT (-77.094591 38.83146),1,1
1,2026,Tina,Allison,Chestnut Academy,CA,1,Female,Asian,Hispanic,5100 Cleveland Street,Virginia Beach,VA,23462,36.843364,-76.159497,"5100 Cleveland Street, Virginia Beach, VA 23462",POINT (-76.159497 36.843364),1,1
2,287,Joseph,Anderson,Chestnut Academy,CA,1,Male,Asian,Non-Hispanic,185 Washington Lee Dr,Bristol,VA,24201,36.611087,-82.178164,"185 Washington Lee Dr, Bristol, VA 24201",POINT (-82.178164 36.611087),1,1
3,1338,Lonnie,Anderson,Chestnut Academy,CA,1,Male,White,Hispanic,5720 Marshall Ave.,Newport News,VA,23605,37.003500,-76.426700,"5720 Marshall Ave., Newport News, VA 23605",POINT (-76.4267 37.0035),1,1
4,1847,Randy,Archer,Chestnut Academy,CA,1,Male,White,Non-Hispanic,8201 Jefferson Davis Hwy,Fredericksburg,VA,22407,38.182400,-77.511400,"8201 Jefferson Davis Hwy, Fredericksburg, VA 2...",POINT (-77.5114 38.1824),1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3362,Elizabeth,Wells,Sycamore Academy,SA,K,Female,African American,Hispanic,7107 Sydenstricker Rd,Springfield,VA,22152,38.762200,-77.243600,"7107 Sydenstricker Rd, Springfield, VA 22152",POINT (-77.2436 38.7622),1,0
3996,2288,Christina,White,Sycamore Academy,SA,K,Female,African American,Hispanic,6915 BRADDOCK RD,SPRINGFIELD,VA,22151,38.810948,-77.185209,"6915 BRADDOCK RD, SPRINGFIELD, VA 22151",POINT (-77.185209 38.810948),1,0
3997,3822,Felicia,Wilkerson,Sycamore Academy,SA,K,Female,White,Hispanic,14175 Turner Drive,Smithfield,VA,23430,36.944300,-76.598200,"14175 Turner Drive, Smithfield, VA 23430",POINT (-76.5982 36.9443),1,0
3998,1677,Jerry,Wilson,Sycamore Academy,SA,K,Male,White,Hispanic,361 Walnut St,Warsaw,VA,22572,37.945528,-76.744655,"361 Walnut St, Warsaw, VA 22572",POINT (-76.744655 37.945528),1,0


In [25]:
df_grad_outcomes = pd.read_csv('grad_outcomes.csv')
df_grad_outcomes

,Student_ID,Starting_Year,Full_School_Name,School,Grade,Gender,Race,Ethnicity,Outcome,Students
0,30000,2018,Sycamore Academy,SA,12,Female,African American,Non-Hispanic,Other/Unknown,1
1,30001,2018,Chestnut Academy,CA,12,Female,Asian,Non-Hispanic,4 Year College,1
2,30002,2018,Sycamore Academy,SA,12,Female,Asian,Non-Hispanic,Other/Unknown,1
3,30003,2018,Hickory Academy,HA,12,Female,White,Non-Hispanic,4 Year College,1
4,30004,2018,Hickory Academy,HA,12,Male,White,Non-Hispanic,4 Year College,1
...,...,...,...,...,...,...,...,...,...,...
1070,31070,2022,Chestnut Academy,CA,12,Male,White,Non-Hispanic,4 Year College,1
1071,31071,2022,Sycamore Academy,SA,12,Male,White,Hispanic,4 Year College,1
1072,31072,2022,Dogwood Academy,DA,12,Male,White,Hispanic,Trade School,1
1073,31073,2022,Chestnut Academy,CA,12,Male,Asian,Non-Hispanic,4 Year College,1


In [11]:
df_test_results = pd.read_csv('test_results.csv')
df_test_results

,Student_ID,School,Grade,Starting_Year,Period,Score
0,42018,CA,1,2023,Fall,47
1,43714,CA,1,2023,Fall,49
2,43215,CA,1,2023,Fall,57
3,43940,CA,1,2023,Fall,63
4,42859,CA,1,2023,Fall,51
...,...,...,...,...,...,...
7995,42179,SA,K,2023,Spring,58
7996,43223,SA,K,2023,Spring,50
7997,42334,SA,K,2023,Spring,60
7998,43592,SA,K,2023,Spring,52


In [12]:
enrollment_comparisons = ['School', 'Grade', 'Gender',
'Race', 'Ethnicity'] 


In [13]:
enrollment_comparisons = ['School', 'Grade', 'Gender',
'Race', 'Ethnicity'] 


def merge_demographics_into_df(df):
    '''This function merges demographic variables from df_current_enrollment
    into the DataFrame passed to df, then returns the new version
    of the DataFrame.'''
    # Creating a copy of df_current_enrollment that only contains 
    # Student IDs (which will serve as the key for the merge) and 
    # the demographic values contained in enrollment_comparisons:
    df_curr_enrollment_for_merge = df_curr_enrollment.copy()[['Student_ID'] + enrollment_comparisons]
    # Some of these demographic values may already be present within 
    # the DataFrame, in which case they should be removed from
    # df_curr_enrollment_for_merge so that we don't end up with multiple
    # copies of the same column.
    for column in df.columns:
        if column in enrollment_comparisons:
            df_curr_enrollment_for_merge.drop(column, axis = 1, inplace = True)
    return df.merge(df_curr_enrollment_for_merge, on = 'Student_ID', how = 'left')

In [14]:
merge_demographics_into_df(df_test_results)

,Student_ID,School,Grade,Starting_Year,Period,Score,Gender,Race,Ethnicity
0,42018,CA,1,2023,Fall,47,NaN,NaN,NaN
1,43714,CA,1,2023,Fall,49,NaN,NaN,NaN
2,43215,CA,1,2023,Fall,57,NaN,NaN,NaN
3,43940,CA,1,2023,Fall,63,NaN,NaN,NaN
4,42859,CA,1,2023,Fall,51,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7995,42179,SA,K,2023,Spring,58,NaN,NaN,NaN
7996,43223,SA,K,2023,Spring,50,NaN,NaN,NaN
7997,42334,SA,K,2023,Spring,60,NaN,NaN,NaN
7998,43592,SA,K,2023,Spring,52,NaN,NaN,NaN


In [15]:
df_curr_enrollment_for_merge = df_curr_enrollment.copy()[['Student_ID'] + enrollment_comparisons]
for column in df_test_results.columns:
    if column in enrollment_comparisons:
        df_curr_enrollment_for_merge.drop(column, axis = 1, inplace = True)
df_curr_enrollment_for_merge
df_test_results.merge(df_curr_enrollment_for_merge, on = 'Student_ID', how = 'left')

,Student_ID,School,Grade,Starting_Year,Period,Score,Gender,Race,Ethnicity
0,42018,CA,1,2023,Fall,47,NaN,NaN,NaN
1,43714,CA,1,2023,Fall,49,NaN,NaN,NaN
2,43215,CA,1,2023,Fall,57,NaN,NaN,NaN
3,43940,CA,1,2023,Fall,63,NaN,NaN,NaN
4,42859,CA,1,2023,Fall,51,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7995,42179,SA,K,2023,Spring,58,NaN,NaN,NaN
7996,43223,SA,K,2023,Spring,50,NaN,NaN,NaN
7997,42334,SA,K,2023,Spring,60,NaN,NaN,NaN
7998,43592,SA,K,2023,Spring,52,NaN,NaN,NaN


In [16]:
list(df_curr_enrollment['Full_School_Name'].unique())

['Chestnut Academy', 'Dogwood Academy', 'Hickory Academy', 'Sycamore Academy']

In [36]:
def create_pivot_for_charts(original_data_source, y_value,
comparison_values, pivot_aggfunc, filter_list = None, 
color_value = None, drop_color_value_from_x_vals = True, 
secondary_differentiator = None, 
drop_secondary_differentiator_from_x_vals = True,
reorder_bars_by = '', reordering_map = {}, debug = False):
    '''original_data_source: The source of the data that will be graphed.

    y_value: The y value to use within the graph.

    comparison_values: A list of values that will be used to pivot the
    DataFrame. These values help determine the level of detail shown in the
    final bar chart. Set this to an empty list ([]) 
    if no comparison values will be used. 

    pivot_aggfunc: the function ('mean', 'sum', 'count', etc.) to be passed
    to the pivot_table() call.

    filter_list: a list of tuples that govern how the DataFrame will be 
    filtered. The first component of each tuple is a column name; the second
    component is a list of values to include.

    color_value and drop_color_value_from_x_vals: 
    color_value specifies the variable to use for a color-based comparison 
    in the final graph.
    In order to represent all of the specified values in the bar chart, 
    the code creates a column describing all (or almost all) of the 
    pivot index variables in the other columns, which then gets fed into 
    the x axis parameter of a histogram. However, if a color value is also 
    specified, *and* drop_color_value_from_x_vals is set to True, this item 
    will not get added into this column, since this
    data will already get represented in the bar chart (by means of the color
    legend). Removing this value helps
    simplify the final chart output.

    secondary_differentiator: Similar to color_value, this is 
    a second variable that will be represented via a chart feature 
    (such as pattern_shape for a bar chart or line_dash for a 
    line chart). Defining this variable allows it to be
    removed from the column that describes the pivot index variables
    so that the graph can be further simplified.

    drop_secondary_differentiator_from_x_vals: Set this to True to remove
    the variable stored in secondary_differentiator from the pivot table
    column that will contain different x values.

    reorder_bars_by and reordering_map: Variables that you can
    use to update the order of the bars in the resulting chart. For instance,
    suppose you want to order the bars by a 'grade' column whose values
    range from K (kindergarten) to 12. If these are stored as strings 
    (which they often will be due to the inclusion of 'K'),
    the first 5 bars will be 1, 10, 11, 12, and 2, and the 
    last bar will be K. (That's because these bars are 
    being treated alphabetically). However, by setting
    reorder_bars_by to 'grade' and reordering_map to {'K':0, '1':1, '2':2, 
    '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, '10':10, '11':11, 
    '12':12, 1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:7, 8:8, 9:9, 10:10, 11:11, 12:12},
    you can instruct the function to (1) create a new order for the grade
    column and then (2) sort the DataFrame based on this new order. This 
    sort operation will in turn reorder the bars so that 'K' comes first
    and '12' comes last.

    Note: If you don't need to update the sort order of the values in the 
    column whose name was passed to reorder_bars_by, simply keep 
    reordering_map as {}.'''

    # Converting 'None' strings to None values:
    if color_value == 'None':
        color_value = None

    if secondary_differentiator == 'None':
        secondary_differentiator = None


    print("Current state of color_value:",color_value, type(color_value))
    print("Current state of secondary_differentiator:",secondary_differentiator, type(secondary_differentiator))


    print("Filter list:",filter_list)
    data_source = original_data_source.copy() # Included to avoid modifying the
    # original DataFrame (although this line may not be necessary)
    
    all_data_value = 'All'

    # The following line goes through each tuple in filter_list and
    # filters the DataFrame based on the values provided there.
    # filter[0] corresponds to a column in the DataFrame, and filter[1]
    # contains a list of which values to keep within that column.
    data_source_filtered = data_source.copy()
    if filter_list != None:
        for filter in filter_list:
           data_source_filtered.query(
            f"{filter[0]} in {filter[1]}", inplace = True)
    if debug == True:
        print("data_source_filtered:",data_source_filtered)

    # The color value must be present within the comparison_values
    # table. If it is not, the following line sets color_value to None.
    if color_value not in comparison_values:
        color_value = None

    # The same holds true for secondary_differentiator.
    if secondary_differentiator not in comparison_values:
        secondary_differentiator = None

    # In order to show comparisons within the final graph, we need to create
    # a table that contains those various comparisons. This function does so
    # using the pivot_table() function within Pandas. The resulting pivot
    # table will have one row for each comparison combination (as long as
    # y value data were present for that combination.)

    # If at least one comparison value was provided, the comparison_values
    # variable will be used as the index for the pivot_table() function. 
    # Otherwise, a new column will be 
    # created (with the same value in every cell), and the pivot_table()
    # function will use this column as its index instead. 
    if len(comparison_values) == 0:
        data_source_filtered[all_data_value] = all_data_value 
        data_source_pivot = data_source_filtered.pivot_table(
            index = all_data_value, values = y_value, 
            aggfunc = pivot_aggfunc).reset_index()
    else:
        data_source_pivot = data_source_filtered.pivot_table(
            index = comparison_values, values = y_value, 
            aggfunc = pivot_aggfunc).reset_index()

    # Next, we need to create x values that reflect the different column
    # values in each row of the pivot table. These x values will then 
    # get passed to the graphing function.
    # The following lines accomplish this by creating a new 
    # data_source_pivot column that contains strings made up of the 
    # values of each of the columns (other than the y value column) 
    # present in the bar chart. The chart will use these strings as 
    # x values when creating the grouped chart. 

    if len(comparison_values) == 0:
        data_descriptor = all_data_value

    else:
        data_descriptor_values = comparison_values.copy()
        
        # We'll now remove the variables stored in
        # color_value and secondary_differentiator from the chart if
        # drop_color_value_from_x_vals and drop_
        if ((color_value != None) & (len(data_descriptor_values) > 1) 
            & (drop_color_value_from_x_vals == True)):
            data_descriptor_values.remove(color_value) 
            # If a value will be assigned a
            # color component in the graph, it doesn't need to be assigned a 
            # group component, since it will show up in the graph regardless. 
            # Removing it here helps simplify the graph.

        # Performing the same steps for secondary_differentiator:
        if ((secondary_differentiator != None) & 
            (len(data_descriptor_values) > 1) 
            & (drop_secondary_differentiator_from_x_vals == True)):
            data_descriptor_values.remove(secondary_differentiator) 
            

        print(data_descriptor_values)   
        data_descriptor = data_source_pivot[
            data_descriptor_values[0]].copy().astype('str') # This line initializes 
            # data_descriptor as the first item within data_descriptor_values.
            # copy() is needed in order to avoid modifying this column when
        # the group column gets chosen.
        # The following for loop iterates through each column name (except
        # for the initial column, which has already been added
        # to data_descriptor) in order to set data_descriptor with all the
        # the values present in data_descriptor_values.
        # The use of a for loop allows this code to adapt to different variable
        # choices and different column counts.
        for i in range(1, len(data_descriptor_values)):
            data_descriptor += ' ' + data_source_pivot[
                data_descriptor_values[i]].astype('str') # This line adds the value of a 
                # given column to data_descriptor.

    data_source_pivot['Group'] = data_descriptor # This group column will be 
    # used as the x value of the histogram.

    # The following code reorders the rows in the pivot table
    # in order to change the order of the bars in the ensuing chart.
    # See the description of reorder_bars_by and reordering_map
    # in the function docstring for more information.
    if (reorder_bars_by != '') & (reorder_bars_by in data_source_pivot.columns):
        # The above line first checks to ensure that the column passed to
        # reorder_bars_by is actually in the pivot; otherwise, we'll run 
        # into an error by trying to sort by a nonexistent column.
        if reordering_map == {}: # Since nothing has been passed to 
            # reordering_map, the function will simply sort the DataFrame
            # by the values in the column referenced by reorder_bars_by.
            data_source_pivot.sort_values('reorder_bars_by', inplace = True)
        else: # In this case, the function will first create a separate column
            # that will store a new order of the values in reorder_bars_by,
            # then sort the DataFrame by that column instead. 
            data_source_pivot['column_for_sorting'] = data_source_pivot[
                reorder_bars_by].map(reordering_map)
            data_source_pivot.sort_values('column_for_sorting', 
            inplace = True)
            data_source_pivot.drop('column_for_sorting', axis = 1, 
            inplace = True) # This column is no longer needed,
            # so we can remove it from the DataFrame.
    
    print("Pivot table created for charts/tables:")
    print(data_source_pivot)
    return data_source_pivot


In [18]:
def create_interactive_bar_chart_and_table(data_source_pivot, y_value,
comparison_values, color_value = None, color_discrete_map = None, 
barmode = 'group', color_discrete_sequence = px.colors.qualitative.Light24,
secondary_differentiator = None, text_auto = True, label_round_precision = None,
table_round_precision = None):
    '''This function converts a pivot table (presumably one returned by
    create_pivot_for_charts() into an interactive bar chart and table.

    data_source_pivot: The pivot table on which the chart will be based.
    It is expected, but not required, that this table originate from
    create_pivot_for_charts().

    For definitions of y_value, comparison_values, and color_value,
    see create_pivot_for_charts().

    color_discrete_map: A custom color mapping to pass to the chart.

    color_discrete_sequence: The color palette to use for the charts.
    The default is Light24 because its use of 24 distinct colors
    helps prevent bar colors from overlapping.

    barmode: The means by which the bars will be presented relative to
    one another. This keyword comes from Plotly's px.histogram() code.

    secondary_differentiator: A value to pass to the pattern_shape argument,
    allowing for further distinction between different bars. For documentation
    on pattern_shape, see:
    # https://plotly.com/python-api-reference/generated/plotly.express.bar

    text_auto: Set to True to show data labels within charts.

    label_round_precision: The extent to which chart labels should be rounded.
    If the label is originally 555.933, a label_round_precision of 1 will
    produce the number 555.9, and a label_round_precision of 0 will produce
    556. No rounding will occur if label_round_precision is set to None.

    table_round_precision: This variable rounds table values in the same way
    that label_round_precision rounds label values.

    '''

    # Converting 'None' strings to None values:
    if color_value == 'None':
        color_value = None

    if secondary_differentiator == 'None':
        secondary_differentiator = None


    data_source_pivot_for_table = data_source_pivot.copy() # This script 
    # will apply changes to copies of data_source_pivot so that the original
    # pivot table is not affected.

    if table_round_precision != None:
        data_source_pivot_for_table[y_value] = round(
            data_source_pivot_for_table[y_value], table_round_precision)

    table_data = data_source_pivot_for_table.to_dict('records') 
    # See https://dash.plotly.com/datatable


    data_source_pivot_for_chart = data_source_pivot.copy()

    # There is no need to perform bar grouping if only one pivot variable 
    # exists, so the following if/else statement sets barmode to 
    # 'relative' in that case. Otherwise, barmode is set to 'group' 
    # in order to simplify the x axis variables.
    if len(comparison_values) == 1:
        selected_barmode = 'relative'
    else:
        selected_barmode = barmode
    
    # The color value must be present within the comparison_values
    # table in order for the code to utilize it. 
    # If it is not, the following line sets color_value to None.
    if color_value not in comparison_values:
        color_value = None

    # The same holds true for secondary_differentiator.
    if secondary_differentiator not in comparison_values:
        secondary_differentiator = None


    # Rounding y values to be shown in labels:
    if label_round_precision != None:
        data_source_pivot_for_chart[y_value] = round(
            data_source_pivot_for_chart[y_value], 
        label_round_precision)

    output_histogram = px.histogram(data_source_pivot_for_chart, x = 'Group', 
    y = y_value, color = color_value, 
    barmode = selected_barmode, color_discrete_map=color_discrete_map,
    color_discrete_sequence=color_discrete_sequence,
    pattern_shape = secondary_differentiator, text_auto = text_auto
    )


    return output_histogram, table_data

In [19]:
def create_interactive_line_chart_and_table(data_source_pivot, y_value, 
comparison_values, color_value = None, color_discrete_map = None, 
color_discrete_sequence = px.colors.qualitative.Light24, 
markers = True, secondary_differentiator = None,
show_labels = True, label_round_precision = None,
table_round_precision = None):
    '''This function converts a pivot table (presumably one returned by
    create_pivot_for_charts() into an interactive line chart and table.

    data_source_pivot: The pivot table on which the chart will be based.
    It is expected, but not required, that this table originate from
    create_pivot_for_charts().

    For definitions of y_value, comparison_values, and color_value,
    see create_pivot_for_charts().

    color_discrete_map: A custom color mapping to pass to the chart.

    color_discrete_sequence: The color palette to use for the charts.
    The default is Light24 because its use of 24 distinct colors
    helps prevent bar colors from overlapping.

    markers: Set to True to add markers to your line chart and False to
    omit them.

    secondary_differentiator: A variable to be passed to the line_dash
    # argument of px.line(). For documentation on line_dash,
    # see: https://plotly.com/python-api-reference/generated/plotly.express.line 

    show_labels: Set to True to show data labels within charts.

    label_round_precision: The extent to which chart labels should be rounded.
    If the label is originally 555.933, a label_round_precision of 1 will
    produce the number 555.9, and a label_round_precision of 0 will produce
    556. No rounding will occur if label_round_precision is set to None.

    table_round_precision: This variable rounds table values in the same way
    that label_round_precision rounds label values.

    '''

    # Converting 'None' strings to None values:
    if color_value == 'None':
        color_value = None

    if secondary_differentiator == 'None':
        secondary_differentiator = None

    # The color value must be present within the comparison_values
    # table in order for the code to utilize it. 
    # If it is not, the following line sets color_value to None.
    if color_value not in comparison_values:
        color_value = None

    # The same holds true for secondary_differentiator.
    if secondary_differentiator not in comparison_values:
        secondary_differentiator = None


    data_source_pivot_for_table = data_source_pivot.copy() # This script 
    # will apply changes to copies of data_source_pivot so that the original
    # pivot table is not affected.
    
    if table_round_precision != None:
        data_source_pivot_for_table[y_value] = round(
            data_source_pivot_for_table[y_value], table_round_precision)

    table_data = data_source_pivot_for_table.to_dict('records') 
    # See https://dash.plotly.com/datatable


    data_source_pivot_for_chart = data_source_pivot.copy()


    # Rounding y values to be shown in labels:
    if label_round_precision != None:
        data_source_pivot_for_chart[y_value] = round(
            data_source_pivot_for_chart[y_value], 
        label_round_precision)

    # Determining whether to show labels:
    if show_labels == True:
        text = y_value
    else:
        text = None

    output_chart = px.line(data_source_pivot_for_chart, x = 'Group', 
    y = y_value, color = color_value,
    color_discrete_map=color_discrete_map,
    color_discrete_sequence=color_discrete_sequence,
    markers = markers, line_dash = secondary_differentiator, text = y_value
    )
    # See https://plotly.com/python/line-charts/

    return output_chart, table_data

In [41]:
grad_outcomes_pivot = create_pivot_for_charts(
    original_data_source=df_grad_outcomes, y_value = 'Students', 
    comparison_values = ['Starting_Year', 'Outcome'] + enrollment_comparisons, pivot_aggfunc= 'sum', 
    filter_list = None, color_value = 'School', 
    secondary_differentiator = None, reorder_bars_by = None, 
    reordering_map = None, debug = True)
grad_outcomes_pivot    

Current state of color_value: School <class 'str'>
Current state of secondary_differentiator: None <class 'NoneType'>
Filter list: None
data_source_filtered:       Student_ID  Starting_Year  Full_School_Name School  Grade  Gender  \
0          30000           2018  Sycamore Academy     SA     12  Female   
1          30001           2018  Chestnut Academy     CA     12  Female   
2          30002           2018  Sycamore Academy     SA     12  Female   
3          30003           2018   Hickory Academy     HA     12  Female   
4          30004           2018   Hickory Academy     HA     12    Male   
...          ...            ...               ...    ...    ...     ...   
1070       31070           2022  Chestnut Academy     CA     12    Male   
1071       31071           2022  Sycamore Academy     SA     12    Male   
1072       31072           2022   Dogwood Academy     DA     12    Male   
1073       31073           2022  Chestnut Academy     CA     12    Male   
1074       31074 

,Starting_Year,Outcome,School,Grade,Gender,Race,Ethnicity,Students,Group
0,2018,2 Year College,CA,12,Female,Asian,Non-Hispanic,1,2018 2 Year College 12 Female Asian Non-Hispanic
1,2018,2 Year College,CA,12,Female,White,Non-Hispanic,1,2018 2 Year College 12 Female White Non-Hispanic
2,2018,2 Year College,CA,12,Male,White,Hispanic,1,2018 2 Year College 12 Male White Hispanic
3,2018,2 Year College,DA,12,Female,Asian,Non-Hispanic,1,2018 2 Year College 12 Female Asian Non-Hispanic
4,2018,2 Year College,DA,12,Male,American Indian,Non-Hispanic,1,2018 2 Year College 12 Male American Indian No...
...,...,...,...,...,...,...,...,...,...
561,2022,Trade School,SA,12,Male,African American,Non-Hispanic,1,2022 Trade School 12 Male African American Non...
562,2022,Trade School,SA,12,Male,American Indian,Hispanic,1,2022 Trade School 12 Male American Indian Hisp...
563,2022,Trade School,SA,12,Male,American Indian,Non-Hispanic,1,2022 Trade School 12 Male American Indian Non-...
564,2022,Trade School,SA,12,Male,White,Hispanic,1,2022 Trade School 12 Male White Hispanic


In [42]:
grad_outcomes_bar_chart, grad_outcomes_table = create_interactive_bar_chart_and_table(
    data_source_pivot = grad_outcomes_pivot, y_value = 'Students', 
    comparison_values = enrollment_comparisons, 
    color_value = 'School', 
    secondary_differentiator= None)

grad_outcomes_bar_chart

In [ ]:
df_results_pivot = create_pivot_for_charts(original_data_source = df_test_results, y_value = 'Score',
comparison_values = ['Period', 'School', 'Grade'], pivot_aggfunc = 'mean', filter_list = None, 
color_value = 'School', drop_color_value_from_x_vals = True, 
reorder_bars_by = '', reordering_map = {}, debug = False, secondary_differentiator = 'Grade')

In [ ]:
test_results_bar_chart, test_results_table = create_interactive_bar_chart_and_table(data_source_pivot = df_results_pivot, y_value = 'Score',
comparison_values = ['Period', 'Grade', 'School'], color_value = 'Grade', color_discrete_map = None, barmode = 'group',
color_discrete_sequence = px.colors.qualitative.Light24, secondary_differentiator = 'School', text_auto = True, label_round_precision = 3, table_round_precision = 3)

test_results_bar_chart

In [ ]:
# test_results_table

In [ ]:
output_chart, table_data = create_interactive_line_chart_and_table(data_source_pivot = df_results_pivot, y_value = 'Score',
comparison_values = ['Period', 'Grade', 'School'], color_value = 'Grade', color_discrete_map = None, color_discrete_sequence = px.colors.qualitative.Light24, secondary_differentiator = 'School', label_round_precision = 2, table_round_precision = 2)

output_chart


In [ ]:
# table_data